<a href="https://colab.research.google.com/github/park-jiha/CNN/blob/main/1.%20MNIST_CNN_%EB%AA%A8%EB%8D%B8_%EC%83%9D%EC%84%B1_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MNIST_CNN 

### GPU 사용을 위한 환경 세팅
- 런타임 => 런타임유형변경 => 하드웨어가속기 => GPU로 설정

### GUI 확인
![GUI](https://user-images.githubusercontent.com/11758940/84726212-581efd00-afc7-11ea-8166-dd4dda7b8e28.png)

### 잠깐! Pytorch Document랑 친해지자
https://pytorch.org/docs/stable/index.html

In [94]:
# 필요 라이브러리 import

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [95]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda device 설정!!

# 시드 고정, 설명 들었었죠?
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [96]:
# MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [97]:
# dataset loader

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=100,
                                          shuffle=True,
                                          drop_last=True)

## >> 버전 1.0
![대체 텍스트](https://user-images.githubusercontent.com/11758940/84732337-63c5f000-afd6-11ea-9e7b-3c4c88785213.png)

In [98]:
## 버전 1.0 = 쭉 나열하기
class CNN_v1(torch.nn.Module):
  def __init__(self):
    super(CNN_v1,self).__init__() # 이부분 빼먹기 쉬운데 조심할 것

    self.conv1 = torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
    self.relu1 = torch.nn.ReLU()
    self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv2 = torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
    self.relu2 = torch.nn.ReLU()
    self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2,stride=2) 

    self.fc = torch.nn.Linear(7*7*64,10,bias=True)
    torch.nn.init.kaiming_uniform(self.fc.weight)

  def forward(self,x):
    out = self.conv1(x)
    out = self.relu1(out)
    out = self.maxpool1(out)
    out = self.conv2(out)
    out = self.relu2(out)
    out = self.maxpool2(out)
    out = out.view(out.size(0),-1)     # out = Batch(0) x 7(1) x 7(2) x 64(3)
    out = self.fc(out)
    return out

In [ ]:
# 반드시 직접 생성해보세요!







## >> 버전 2.0
![대체 텍스트](https://user-images.githubusercontent.com/11758940/84732337-63c5f000-afd6-11ea-9e7b-3c4c88785213.png)

In [99]:
# 버전 2.0 = layer별로 묶어주기
class CNN_v2(torch.nn.Module):
  def __init__(self):
    super(CNN_v2,self).__init__() # 이부분 빼먹기 쉬운데 조심할 것

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    ) 

    self.fc = torch.nn.Linear(7*7*64,10,bias=True)
    torch.nn.init.kaiming_uniform(self.fc.weight)

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0),-1)     # out = Batch x 7 x 7 x 64
    out = self.fc(out)
    return out

In [ ]:
# 반드시 직접 생성해보세요!








In [100]:
# 본인이 편하다고 생각하는 방식으로 쓰면 됩니다. 정답은 없어요

model = CNN_v1().to(device)
# model = CNN_v2().to(device)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  from ipykernel import kernelapp as app


In [101]:
# MNIST는 다중분류 -> CrossEntropyLoss겠죠?
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [102]:
# parameters -> 여러분들이 실험적으로 선택하는 파라미터 값들입니다. 본인의 생각대로 여러번 실험해보고 최적의 값을 찾아보세요!

learning_rate = 0.1
training_epochs = 12

In [103]:
# 2D => 1D로 변경할 필요 없음, 왜냐? CNN이기 때문에!

total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        X = X.to(device)
        Y = Y.to(device)

        # H(x) 계산
        hypothesis = model(X)
        # cost 계산
        cost = criterion(hypothesis, Y)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>2}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

[Epoch:  1] cost = 0.25607422
[Epoch:  2] cost = 0.0825096592
[Epoch:  3] cost = 0.0635287762
[Epoch:  4] cost = 0.054260537
[Epoch:  5] cost = 0.0461570807
[Epoch:  6] cost = 0.0413110517
[Epoch:  7] cost = 0.0378277488
[Epoch:  8] cost = 0.0342775248
[Epoch:  9] cost = 0.0318631157
[Epoch: 10] cost = 0.0289999284
[Epoch: 11] cost = 0.0266362764
[Epoch: 12] cost = 0.024487555
Learning Finished!


In [104]:
# 2D => 1D로 변경할 필요 없음

# 테스트 후 정확도 확인
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    # 예측
    prediction = model(X_test)

    correct_prediction = torch.argmax(prediction, 1) == Y_test  # argmax를 쓰는 이유? = 가장 큰 가중치를 갖는 값을 뽑아온다고 생각하면 된다. Softmax!
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9837999939918518


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
